In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

/kaggle/input/chat-gpt-rap-ai/Chat GPT Rap Data.txt
/kaggle/input/data-1/Chat GPT Rap Data.txt


In [2]:
# load ascii text and covert to lowercase
filename = "/kaggle/input/data-1/Chat GPT Rap Data.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [3]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [4]:
print(list(char_to_int.keys()))

['\n', ' ', "'", ',', '-', '.', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  15036
Total Vocab:  33


In [6]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  14936


In [7]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)

In [8]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [9]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [10]:
model.fit(X, y, epochs=40, batch_size=128, callbacks=callbacks_list)

Epoch 1/40
117/117 [==============================] - ETA: 0s - loss: 3.0915
Epoch 1: loss improved from inf to 3.09146, saving model to weights-improvement-01-3.0915.hdf5
117/117 [==============================] - 15s 30ms/step - loss: 3.0915
Epoch 2/40
117/117 [==============================] - ETA: 0s - loss: 3.0493
Epoch 2: loss improved from 3.09146 to 3.04932, saving model to weights-improvement-02-3.0493.hdf5
117/117 [==============================] - 3s 29ms/step - loss: 3.0493
Epoch 3/40
117/117 [==============================] - ETA: 0s - loss: 3.0384
Epoch 3: loss improved from 3.04932 to 3.03845, saving model to weights-improvement-03-3.0384.hdf5
117/117 [==============================] - 3s 30ms/step - loss: 3.0384
Epoch 4/40
117/117 [==============================] - ETA: 0s - loss: 2.9711
Epoch 4: loss improved from 3.03845 to 2.97112, saving model to weights-improvement-04-2.9711.hdf5
117/117 [==============================] - 3s 29ms/step - loss: 2.9711
Epoch 5/40
117/

In [11]:
import sys

# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
# pattern = ""
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" killed navigator,
my flow's so smooth, it's like a sweet sedator.
i bring the fire, ignite the stage "
 with my presence,
when i'm on the mic, i leave an indelible essence.

i'm the rap phenomenon, the wordsmith extraordinaire,
i drop rhymes so sick, it's like a sonic warfare.
from the east to the west, i'm known coast to coast,
i'm the rap game's heavyweight, i boast and i toast.

i'm the rap phenomenon, the wordsmith extraordinaire,
i drop rhymes so sick, it's like a sonic warfare.
from the east to the west, i'm known coast to coast,
i'm the rap game's heavyweight, i boast and i toast.

i'm the rap phenomenon, the wordsmith extraordinaire,
i drop rhymes so sick, it's like a sonic warfare.
from the east to the west, i'm known coast to coast,
i'm the rap game's heavyweight, i boast and i toast.

i'm the rap phenomenon, the wordsmith extraordinaire,
i drop rhymes so sick, it's like a sonic warfare.
from the east to the west, i'm known coast to coast,
i'm the rap game's heavyweig